In [42]:
from pathlib import Path

import pandas as pd

In [43]:
LOG_PATH = Path('..', 'log').with_suffix('.jsonl')

In [44]:
run_path = Path('/Users/njf/code/frankie/eval/runs/2025-05-05T22:52:48.jsonl')
with open(run_path) as run_file:
    data = pd.read_json(run_file, lines=True)

In [45]:
row = data.sample(1)

In [46]:
row['messages'].to_dict()

{8: [{'role': 'system',
   'content': "You are a helpful assistant tasked with answering questions that require multiple intermediate steps of reasoning to arrive at a final answer.\n\nThe questions involve using World Bank data for various countries and indicators, and you have access to a set of tools that can help you retrieve and process this data.\n\nStep-by-step, use the tools provided to execute that plan. Use the `think` tool to think aloud about the steps you will take to solve the problem. Use the `final_answer` tool to return your final answer.\n\nUse the results of each tool call to inform your next step. Note that passing tool calls as the arguments to other tool calls is not allowed. Instead, execute each tool call separately and use the results to perform subsequent calls.\n\nIf a tool call fails, use the error message to help you debug the issue, re-plan, and try again if possible.\n\nBear in mind that some data may not be available for certain countries, indicators, or

In [49]:
def get_model_final_answer(row):
    # Get the final answer from the row
    messages = row['messages'].to_dict()[row.index[0]]
    # Get the messages where 'role' = 'tool' and 'name' = 'final_answer'
    final_answer = [msg for msg in messages if msg['role'] == 'tool' and msg['name'] == 'final_answer']
    if final_answer:
        # Get the content of the final answer
        final_answer_content = final_answer[0]['content']
        # Convert the content to a string
        final_answer_str = str(final_answer_content)
        return final_answer_str
    else:
        # If no final answer is found, return None
        return None

def get_gold_final_answer(row):
    # Get the final answer from the actions sequence
    actions = row['actions'].to_dict()[row.index[0]]
    # Get the actions where 'name' = 'final_answer'
    final_answer = [action for action in actions if action['name'] == 'final_answer']
    return final_answer['result']

def get_model_tool_calls(row):
    # Get the tool calls from the row
    messages = row['messages'].to_dict()[row.index[0]]
    # Get the messages where 'role' = 'tool'
    tool_calls = [msg['tool_calls'][0]['function'] for msg in messages if msg['role'] == 'assistant' and 'tool_calls' in msg]
    return tool_calls

def get_gold_tool_calls(row):
    # Get the tool calls from the actions sequence
    actions = row['actions'].to_dict()[row.index[0]]
    return actions